In [21]:
from docx2pdf import convert
import os
import pandas as pd


In [3]:
# colocar aqui la ruta de los archivos
path = r'C:\Users\bismarksr\Desktop\PLANOS'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 2 INFORMES
2 INFORMES


['C:\\Users\\bismarksr\\Desktop\\PLANOS/PROPIEDAD_17.pdf',
 'C:\\Users\\bismarksr\\Desktop\\PLANOS/PROPIEDAD_30.pdf']

In [5]:
# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [29]:
contenido = os.listdir(path)
dic_files = {}
for file in contenido:
    split1 = file.split('.')    
    split2 = split1[0].split('_')
    dic_files[str(split2[1])] = file
print(len(dic_files), 'informes')
dic_files

2 informes


{'17': 'PROPIEDAD_17.pdf', '30': 'PROPIEDAD_30.pdf'}

In [30]:
# CARGAR LOS PDFS A GOOGLE DRIVE
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "PLANOS ANOMALIAS"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(path + r'/' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file PROPIEDAD_17.pdf with mimeType application/pdf
Created file PROPIEDAD_30.pdf with mimeType application/pdf
end


In [31]:
dic_urls

{'17': 'https://drive.google.com/file/d/1G__TP19oel2o4MNLlgRSnsxgFlGSzhMJ/view?usp=drivesdk',
 '30': 'https://drive.google.com/file/d/193OE-0wsSQ6UZctoZXGzVH3Q673W1n7d/view?usp=drivesdk'}

In [35]:
df = pd.DataFrame(list(dic_urls.items()), columns=['ID', 'URL'])

# Exportamos el DataFrame a un archivo Excel
df.to_excel('ruta_planos.xlsx', index=False)